In [27]:
#Importaciones
import problema_espacio_estados as probee
import búsqueda_espacio_estados as búsqee
import copy
from tkinter import *

In [28]:
#Clase que forma la matriz. La matriz es una lista de listas. Cada una de esas listas es una fila de la matriz.
class Hitori:
    def __init__(self, casillas):
        self.casillas = casillas
    
    #Devuelve la longuitud de la primera fila (que siempre va a existir), representando el número de columnas de la matriz.
    def tamaño_hor(self):
        return len(self.casillas[0])
    
    #Devuelve la longuitud de la lista que forma la matriz, representando el número de filas de la matriz.
    def tamaño_ver(self):
        return len(self.casillas)
    
    #Devuleve el valor numérico de la matriz en la posición fc, que se le pasa como atributo.
    def valor_casilla(self, f, c):
        return self.casillas[f][c]    
    
    #Al imprimit la clase por pantalla, se le da un formato, que es una lista de listas en una sola línea.
    def __str__(self):
        cadena = str(self.casillas)
        
        return cadena

In [29]:
#Subclase de la clase Acción, que ultilizamos para definir las restricciones del juego y para aplicar las acciones.
class Poner_negro(probee.Acción):
    def __init__(self, i, j):
        nombre = 'Posición {},{}'.format(i, j)
        super().__init__(nombre)
        self.f = i
        self.c = j
    
    #Comprueba si el valor de la matriz en la posición fc es cero, es decir que esa posición esta tachada.
    def es_negro(self, estado, f, c):
        return bool(Hitori.valor_casilla(estado,f,c) == 0)
    
    #Comprueba que si a la izquierda, derecha, arriba o abajo de la posición fc de estado existe un cero, por lo que al 
    #tachar la posición fc supondría que exite dos número tachados seguidos, saltándose uan restricción del juego.
    def negro_colindante(self, estado):
        arriba = False
        abajo = False
        izquierda = False
        derecha = False
        
        #No puede existir un cero abajo si estamos en la última fila.
        if self.f < (Hitori.tamaño_ver(estado)-1):
            abajo = self.es_negro(estado, (self.f)+1, self.c)
         
        #No puede existir un cero arriba si estamos en la primera fila.
        if self.f > 0:
            arriba = self.es_negro(estado, (self.f)-1, self.c)
           
        #No puede existir un cero a la derecha si estamos en la última columna.
        if self.c < (Hitori.tamaño_hor(estado)-1):
            derecha = self.es_negro(estado, self.f, (self.c)+1)
            
        #No puede existir un cero a la izquierda estamos en la primera columna.    
        if self.c > 0:
            izquierda = self.es_negro(estado, self.f, (self.c)-1)
                
        return arriba or abajo or izquierda or derecha
    
    #Comprueba que se repite el valor de la posición fc del estado, en la misma fila 
    def duplicado_fila(self, estado):
        tamaño = Hitori.tamaño_hor(estado)
        limite_izq = self.c
        limite_der = tamaño
        #Bucle de [0,c)
        for i in range(0, limite_izq):
            #No queremos comprobar si el cero está duplicado
            if Hitori.valor_casilla(estado, self.f,self.c) != 0:
                #Comprobamos si hay duplicados
                if Hitori.valor_casilla(estado, self.f, i) == Hitori.valor_casilla(estado, self.f, self.c):
                    return True
        
        #Bucle de [c+1,tamaño)
        for j in range((self.c)+1, limite_der):
            #No queremos comprobar si el cero está duplicado
            if Hitori.valor_casilla(estado, self.f,self.c) != 0:
                #Comprobamos si hay duplicados
                 if Hitori.valor_casilla(estado, self.f, j) == Hitori.valor_casilla(estado, self.f, self.c):
                    return True
                
    
    #Comprueba que se repite el valor de la posición fc del estado, en la misma columna 
    def duplicado_columna(self, estado):
        tamaño = Hitori.tamaño_ver(estado)
        limite_arr = self.f
        limite_aba = tamaño
        #Bucle de [0,f)
        for i in range(0, limite_arr):
            #No queremos comprobar si el cero está duplicado
            if Hitori.valor_casilla(estado, self.f,self.c) != 0:
                #Comprobamos si hay duplicados
                if Hitori.valor_casilla(estado, i, self.c) == Hitori.valor_casilla(estado, self.f, self.c):
                    return True
        #Bucle [f+1,tamaño)        
        for j in range((self.f)+1, limite_aba):
            #No queremos comprobar si el cero está duplicado
            if Hitori.valor_casilla(estado, self.f,self.c) != 0:
                #Comprobamos si hay duplicados
                 if Hitori.valor_casilla(estado, j, self.c) == Hitori.valor_casilla(estado, self.f, self.c):
                    return True
            
    
    #
    #
    #          ------------------------
    #          |       |  0  |        |
    #          ------------------------
    #          |    0  |  y  |  0     |
    #          -----------------------
    #          |       |  x  |        |
    #          -----------------------
    #
    #Comprobamos que al poner "x" (la posicón fc del estado), el número "y" se queda encerrado. 
    #
    #
    def abajo_encierra(self, estado):
        arriba = True
        arriba_derecha = True
        arriba_izquierda = True
        
        #Comprobamos que no estamos en la primera columna y/o que no estamos en la primera fila.
        if self.f > 0 and self.c > 0:
            if not self.es_negro(estado, (self.f)-1, (self.c)-1):
                arriba_izquierda = False        
               
        #Comprobamos que no estamos en la última columna y/o que no estamos en la primera fila.    
        if self.f > 0 and self.c < (Hitori.tamaño_hor(estado)-1):      
            if not self.es_negro(estado, (self.f)-1, (self.c)+1):
                arriba_derecha = False
             
        #Comprobamos que no estamos ni en la primera ni en la segunda fila.    
        if self.f > 1: 
            if not self.es_negro(estado, (self.f)-2, self.c):
                arriba = False
            
        #Comprobamos que estamos en la primera fila.    
        if self.f == 0:
            arriba = False
            arriba_derecha = False
            arriba_izquierda = False
            
            
        return arriba and arriba_derecha and arriba_izquierda    
        
    #
    #
    #           ------------------------
    #          |       |  x  |        |
    #          ------------------------
    #          |    0  |  y  |  0     |
    #          -----------------------
    #          |       |  0  |        |
    #          -----------------------
    #
    #Comprobamos que al poner "x" (la posicón fc del estado), el número "y" se queda encerrado.
    #
    #
    def arriba_encierra(self, estado):
        abajo = True
        abajo_derecha = True
        abajo_izquierda = True
        
        #Comprobamos que no estamos en la primera columna y/o que no estamos en la última fila.
        if self.f < Hitori.tamaño_ver(estado)-1 and self.c > 0:
            if not self.es_negro(estado, (self.f)+1, (self.c)-1):
                abajo_izquierda = False        
        
        #Comprobamos que no estamos en la última columna y/o que no estamos en la última fila.
        if self.f < (Hitori.tamaño_ver(estado)-1) and self.c < (Hitori.tamaño_hor(estado)-1):      
            if not self.es_negro(estado, (self.f)+1, (self.c)+1):
                abajo_derecha = False
        
        #Comprobamos que no estamos ni en la última ni en la penúltima fila.
        if self.f < (Hitori.tamaño_ver(estado)-2): 
            if not self.es_negro(estado, (self.f)+2, self.c):
                abajo = False
               
        #Compramos que estamos en la última fila.    
        if self.f == (Hitori.tamaño_ver(estado)-1):
            abajo = False
            abajo_derecha = False
            abajo_izquierda = False
            
            
        return abajo and abajo_derecha and abajo_izquierda    
      
    #
    #
    #           ------------------------
    #          |       |  0  |        |
    #          ------------------------
    #          |    0  |  y  |  x     |
    #          -----------------------
    #          |       |  0  |        |
    #          -----------------------
    #
    #Comprobamos que al poner "x" (la posicón fc del estado), el número "y" se queda encerrado.
    #
    #
    def derecha_encierra(self, estado):
        izquierda = True
        arriba = True
        abajo = True
        
        #Comprobamos que no estamos en la primera columna y/o que no estamos en la primera fila.
        if self.f > 0 and self.c > 0:
            if not self.es_negro(estado, (self.f)-1, (self.c)-1):
                arriba = False        
        
        #Comprobamos que no estamos en la primera columna y/o que no estamos en la última fila.
        if self.f < (Hitori.tamaño_ver(estado)-1) and self.c > 0:      
            if not self.es_negro(estado, (self.f)+1, (self.c)-1):
                abajo = False
                
        #Comprobamos que no estamos ni en la primera ni en la segunda columna.        
        if self.c > 1: 
            if not self.es_negro(estado, (self.f), self.c-2):
                izquierda = False
        
        #Compramos que estamos en la primera columna.
        if self.c == 0:
            izquierda = False
            arriba = False
            abajo = False
            
            
        return izquierda and arriba and abajo  
    
    #
    #
    #           ------------------------
    #          |       |  0  |        |
    #          ------------------------
    #          |    x  |  y  |  0     |
    #          -----------------------
    #          |       |  0  |        |
    #          -----------------------
    #
    #Comprobamos que al poner "x" (la posicón fc del estado), el número "y" se queda encerrado.
    #
    #
    def izquierda_encierra(self, estado):
        derecha = True
        arriba = True
        abajo = True
        
        #Comprobamos que no estamos en la última columna y/o que no estamos en la primera fila.
        if self.f > 0 and self.c < Hitori.tamaño_hor(estado)-1:
            if not self.es_negro(estado, (self.f)-1, (self.c)+1):
                arriba = False        
        
        #Comprobamos que no estamos en la última columna y/o que no estamos en la última fila.
        if self.f < (Hitori.tamaño_ver(estado)-1) and self.c < (Hitori.tamaño_hor(estado)-1):      
            if not self.es_negro(estado, (self.f)+1, (self.c)+1):
                abajo = False
        
        #Comprobamos que no estamos ni en la última ni en la penúltima columna.
        if self.c < Hitori.tamaño_hor(estado)-2: 
            if not self.es_negro(estado, (self.f), self.c+2):
                derecha = False
        
        #Compramos que estamos en la última columna.
        if self.c == (Hitori.tamaño_hor(estado)-1):
            derecha = False
            arriba = False
            abajo = False
            
            
        return derecha and arriba and abajo    
     
        
    #Si se cumple alguna de las restricciones, poner negro la casilla provoca de se encierre una casilla
    def encierra(self, estado):
        return bool(self.abajo_encierra(estado) 
                    or self.arriba_encierra(estado) 
                    or self.izquierda_encierra(estado) 
                    or self.derecha_encierra(estado))
    
    #Comprobamos todas la restricciones, con los métodos anteriores.
    def es_aplicable(self, estado):
        return bool(not self.negro_colindante(estado)
                    and (self.duplicado_fila(estado) or self.duplicado_columna(estado))
                    and not self.encierra(estado))
        
    #Mismo método que duplicado_fila pero pasandole además f y c como atributo.    
    def duplicado_fila_aux(self, estado, f, c):
        num = Hitori.valor_casilla(estado, f, c)
        if estado.casillas[f].count(num) > 1:
            return True
                
    #Mismo método que duplicado_columna pero pasandole además f y c como atributo.
    def duplicado_columna_aux(self, estado, f, c):
        tamaño = Hitori.tamaño_ver(estado)
        limite_izq = f
        limite_der = tamaño
        for i in range(0, limite_izq):
            if Hitori.valor_casilla(estado, f, c) != 0:
                if Hitori.valor_casilla(estado, i, c) == Hitori.valor_casilla(estado, f, c):
                    return True
                
        for j in range(f+1, limite_der):
            if Hitori.valor_casilla(estado, f, c) != 0:
                 if Hitori.valor_casilla(estado, j, c) == Hitori.valor_casilla(estado, f, c):
                    return True
    
    #Mismo método que negro_colindante pero pasandole además f y c como atributo.
    def negro_colindante_aux(self, estado, f, c):
        arriba = False
        abajo = False
        izquierda = False
        derecha = False
        
        if f < (Hitori.tamaño_ver(estado)-1):
            abajo = self.es_negro(estado, f+1, c)
         
        if f > 0:
            arriba = self.es_negro(estado, f-1, c)
            
        if c < (Hitori.tamaño_hor(estado)-1):
            derecha = self.es_negro(estado, f, c+1)
            
        if c > 0:
            izquierda = self.es_negro(estado, f, c-1)
            
        return arriba or abajo or izquierda or derecha
    
    #Mismo método que abajo_encierra pero pasandole además f y c como atributo.
    def abajo_encierra_aux(self, estado, f, c):
        arriba = True
        arriba_derecha = True
        arriba_izquierda = True
        
        if f > 0 and c > 0:
            if not self.es_negro(estado, f-1, c-1):
                arriba_izquierda = False        
                
        if f > 0 and c < (Hitori.tamaño_hor(estado)-1):      
            if not self.es_negro(estado, f-1, c+1):
                arriba_derecha = False
                
        if f > 1: 
            if not self.es_negro(estado, f-2, c):
                arriba = False
                
        if f == 0:
            arriba = False
            arriba_derecha = False
            arriba_izquierda = False
            
            
        return arriba and arriba_derecha and arriba_izquierda
    
    #Mismo método que arriba_encierra pero pasandole además f y c como atributo.
    def arriba_encierra_aux(self, estado, f, c):
        abajo = True
        abajo_derecha = True
        abajo_izquierda = True
        
        if f < Hitori.tamaño_ver(estado)-1 and c > 0:
            if not self.es_negro(estado, f+1, c-1):
                abajo_izquierda = False        
                
        if f < (Hitori.tamaño_ver(estado)-1) and c < (Hitori.tamaño_hor(estado)-1):      
            if not self.es_negro(estado, f+1, c+1):
                abajo_derecha = False
                
        if f < (Hitori.tamaño_ver(estado)-2): 
            if not self.es_negro(estado, f+2, c):
                abajo = False
                
        if f == (Hitori.tamaño_ver(estado)-1):
            abajo = False
            abajo_derecha = False
            abajo_izquierda = False
            
            
        return abajo and abajo_derecha and abajo_izquierda    
      
    #Mismo método que derecha_encierra pero pasandole además f y c como atributo.
    def derecha_encierra_aux(self, estado, f, c):
        izquierda = True
        arriba = True
        abajo = True
        
        if f > 0 and c > 0:
            if not self.es_negro(estado, f-1, c-1):
                arriba = False        
                
        if f < (Hitori.tamaño_ver(estado)-1) and c > 0:      
            if not self.es_negro(estado, f+1, c-1):
                abajo = False
                
        if c > 1: 
            if not self.es_negro(estado, f, c-2):
                izquierda = False
                
        if c == 0:
            izquierda = False
            arriba = False
            abajo = False
            
            
        return izquierda and arriba and abajo  
    
    #Mismo método que izquierda_encierra pero pasandole además f y c como atributo.
    def izquierda_encierra_aux(self, estado, f, c):
        derecha = True
        arriba = True
        abajo = True
        
        if f > 0 and c < Hitori.tamaño_hor(estado)-1:
            if not self.es_negro(estado, f-1, c+1):
                arriba = False        
                
        if f < (Hitori.tamaño_ver(estado)-1) and c < (Hitori.tamaño_hor(estado)-1):      
            if not self.es_negro(estado, f+1, c+1):
                abajo = False
                
        if c < Hitori.tamaño_hor(estado)-2: 
            if not self.es_negro(estado, f, c+2):
                derecha = False
                
        if c == (Hitori.tamaño_hor(estado)-1):
            derecha = False
            arriba = False
            abajo = False
            
            
        return derecha and arriba and abajo 
    
    #Mismo método que encierra pero pasandole además f y c como atributo.
    def encierra_aux(self, estado, f, c):
        return bool(self.abajo_encierra_aux(estado, f, c) 
                    or self.arriba_encierra_aux(estado, f, c) 
                    or self.izquierda_encierra_aux(estado, f, c) 
                    or self.derecha_encierra_aux(estado, f, c))
    
    #Método que realiza la acción de poner en negro una casilla.
    #Las posiciones que se encuentran a la izquierda, derecha, arriba y abajo de la posición que ponemos a cero no pueden ser 
    #cero también, entonces comprobamos si estos números están duplicados en fila y/o columna. En ese caso ponemos a cerso lo que 
    #están duplicados, si se puede. Si no se puede no es solución.
    def aplicar(self, estado):
        nuevo_estado = copy.deepcopy(estado)
        nuevo_estado.casillas[self.f][self.c] = 0
        
        #Comprobamos que existe una posición a la derecha
        if self.c < (Hitori.tamaño_hor(estado)-1):
            #Comprobamos que hay duplicados en la fila
            if self.duplicado_fila_aux(estado, self.f, (self.c)+1):
                tamaño = Hitori.tamaño_hor(estado)
                #No llegaría al que ponemos en negro
                limite_izq = self.c
                limite_der = tamaño
                for i in range(0, limite_izq):
                    if (Hitori.valor_casilla(estado, self.f, i) == Hitori.valor_casilla(estado, self.f, self.c+1)
                       and not self.negro_colindante_aux(nuevo_estado, self.f, i)
                       and not self.encierra_aux(nuevo_estado, self.f, i)):
                        nuevo_estado.casillas[self.f][i] = 0

                for j in range((self.c)+2, limite_der):
                    if (Hitori.valor_casilla(estado, self.f, j) == Hitori.valor_casilla(estado, self.f, self.c+1)
                       and not self.negro_colindante_aux(nuevo_estado, self.f, j)
                       and not self.encierra_aux(nuevo_estado, self.f, j)):
                        nuevo_estado.casillas[self.f][j] = 0
            #Comprobamos que hay duplicados en columna
            if self.duplicado_columna_aux(estado, self.f, (self.c)+1):
                tamaño = Hitori.tamaño_ver(estado)
                limite_arr = self.f
                limite_aba = tamaño
                for i in range(0, limite_arr):
                    if (Hitori.valor_casilla(estado, i, (self.c)+1) == Hitori.valor_casilla(estado, self.f, self.c+1)
                       and not self.negro_colindante_aux(nuevo_estado, i, self.c+1)
                       and not self.encierra_aux(nuevo_estado, i, self.c+1)):
                        nuevo_estado.casillas[i][self.c+1] = 0

                for j in range((self.f)+1, limite_aba):
                    if (Hitori.valor_casilla(estado, j, (self.c)+1) == Hitori.valor_casilla(estado, self.f, self.c+1)
                       and not self.negro_colindante_aux(nuevo_estado, j, self.c+1)
                       and not self.encierra_aux(nuevo_estado, j, self.c+1)):
                        nuevo_estado.casillas[j][self.c+1] = 0
                        
        #Comprobamos que existe una posición a la izquierda
        if self.c > 0:
            #Comprobamos que hay duplicados en la fila
            if self.duplicado_fila_aux(estado, self.f, (self.c)-1):
                tamaño = Hitori.tamaño_hor(estado)
                #No llegaría al que ponemos en negro
                limite_izq = self.c
                limite_der = tamaño
                for i in range(0, limite_izq-1):
                    if (Hitori.valor_casilla(estado, self.f, i) == Hitori.valor_casilla(estado, self.f, self.c-1)
                       and not self.negro_colindante_aux(nuevo_estado, self.f, i)
                       and not self.encierra_aux(nuevo_estado, self.f, i)):
                        nuevo_estado.casillas[self.f][i] = 0

                for j in range((self.c)+1, limite_der):
                    if (Hitori.valor_casilla(estado, self.f, j) == Hitori.valor_casilla(estado, self.f, self.c-1)
                       and not self.negro_colindante_aux(nuevo_estado, self.f, j)
                       and not self.encierra_aux(nuevo_estado, self.f, j)):
                        nuevo_estado.casillas[self.f][j] = 0
            #Comprobamos que hay duplicados en columna
            if self.duplicado_columna_aux(estado, self.f, (self.c)-1):
                tamaño = Hitori.tamaño_ver(estado)
                limite_arr = self.f
                limite_aba = tamaño
                for i in range(0, limite_arr):
                    if (Hitori.valor_casilla(estado, i, (self.c)-1) == Hitori.valor_casilla(estado, self.f, self.c-1)
                       and not self.negro_colindante_aux(nuevo_estado, i, self.c-1)
                       and not self.encierra_aux(nuevo_estado, i, self.c-1)):
                        nuevo_estado.casillas[i][self.c-1] = 0

                for j in range((self.f)+1, limite_aba):
                    if (Hitori.valor_casilla(estado, j, (self.c)-1) == Hitori.valor_casilla(estado, self.f, self.c-1)
                       and not self.negro_colindante_aux(nuevo_estado, j, self.c-1)
                       and not self.encierra_aux(nuevo_estado, j, self.c-1)):
                        nuevo_estado.casillas[j][self.c-1] = 0
                        
        #Comprobamos que existe una posición de arriba
        if self.f > 0:
            #Comprobamos que hay duplicados en la fila
            if self.duplicado_fila_aux(estado, (self.f)-1, self.c):
                tamaño = Hitori.tamaño_ver(estado)
                #No llegaría al que ponemos en negro
                limite_izq = self.c
                limite_der = tamaño
                for i in range(0, limite_izq):
                    if (Hitori.valor_casilla(estado, (self.f)-1, i) == Hitori.valor_casilla(estado, (self.f)-1, self.c)
                       and not self.negro_colindante_aux(nuevo_estado, (self.f)-1, i)
                       and not self.encierra_aux(nuevo_estado, (self.f)-1, i)):
                        nuevo_estado.casillas[self.f-1][i] = 0

                for j in range((self.c)+1, limite_der):
                    if (Hitori.valor_casilla(estado, (self.f)-1, j) == Hitori.valor_casilla(estado, (self.f)-1, self.c)
                       and not self.negro_colindante_aux(nuevo_estado, (self.f)-1, j)
                       and not self.encierra_aux(nuevo_estado, (self.f)-1, j)):
                        nuevo_estado.casillas[self.f-1][j] = 0
            #Comprobamos que hay duplicados en columna
            if self.duplicado_columna_aux(estado, (self.f)-1, self.c):
                tamaño = Hitori.tamaño_ver(estado)
                limite_arr = self.f
                limite_aba = tamaño
                for i in range(0, limite_arr-1):
                    if (Hitori.valor_casilla(estado, i, self.c) == Hitori.valor_casilla(estado, (self.f)-1, self.c)
                       and not self.negro_colindante_aux(nuevo_estado, i, self.c)
                       and not self.encierra_aux(nuevo_estado, i, self.c)):
                        nuevo_estado.casillas[i][self.c] = 0

                for j in range((self.f)+1, limite_aba):
                    if (Hitori.valor_casilla(estado, j, self.c) == Hitori.valor_casilla(estado, (self.f)-1, self.c)
                       and not self.negro_colindante_aux(nuevo_estado, j, self.c)
                       and not self.encierra_aux(nuevo_estado, j, self.c)):
                        nuevo_estado.casillas[j][self.c] = 0
                        
        #Comprobamos que existe una posición de abajo
        if self.f < Hitori.tamaño_ver(estado)-1:
            #Comprobamos que hay duplicados en la fila
            if self.duplicado_fila_aux(estado, (self.f)+1, self.c):
                tamaño = Hitori.tamaño_ver(estado)
                #No llegaría al que ponemos en negro
                limite_izq = self.c
                limite_der = tamaño
                for i in range(0, limite_izq):
                    if (Hitori.valor_casilla(estado, (self.f)+1, i) == Hitori.valor_casilla(estado, (self.f)+1, self.c)
                       and not self.negro_colindante_aux(nuevo_estado, (self.f)+1, i)
                       and not self.encierra_aux(nuevo_estado, (self.f)+1, i)):
                        nuevo_estado.casillas[self.f+1][i] = 0

                for j in range((self.c)+1, limite_der):
                    if (Hitori.valor_casilla(estado, (self.f)+1, j) == Hitori.valor_casilla(estado, (self.f)+1, self.c)
                       and not self.negro_colindante_aux(nuevo_estado, (self.f)+1, j)
                       and not self.encierra_aux(nuevo_estado, (self.f)+1, j)):
                        nuevo_estado.casillas[self.f+1][j] = 0
            #Comprobamos que hay duplicados en columna
            if self.duplicado_columna_aux(estado, (self.f)+1, self.c):
                tamaño = Hitori.tamaño_ver(estado)
                limite_arr = self.f
                limite_aba = tamaño
                for i in range(0, limite_arr):
                    if (Hitori.valor_casilla(estado, i, self.c) == Hitori.valor_casilla(estado, (self.f)+1, self.c)
                       and not self.negro_colindante_aux(nuevo_estado, i, self.c)
                       and not self.encierra_aux(nuevo_estado, i, self.c)):
                        nuevo_estado.casillas[i][self.c] = 0

                for j in range((self.f)+2, limite_aba):
                    if (Hitori.valor_casilla(estado, j, self.c) == Hitori.valor_casilla(estado, (self.f)+1, self.c)
                       and not self.negro_colindante_aux(nuevo_estado, j, self.c)
                       and not self.encierra_aux(nuevo_estado, j, self.c)):
                        nuevo_estado.casillas[j][self.c] = 0    
                        
            
        return nuevo_estado

In [30]:
#Subclase de la clase ProblemaEspacioEstados
class ProblemaHitori(probee.ProblemaEspacioEstados):
    def __init__(self, estado_inicial):
        acciones = [Poner_negro (i,j) for i in range(0, Hitori.tamaño_hor(estado_inicial)) for j in range(0, Hitori.tamaño_ver(estado_inicial))]
        estado_inicial = estado_inicial
        super().__init__(acciones, estado_inicial)
    
    #Comprueba si existe algún duplicado en fila.
    def duplicado_fila(self, estado):
        
        #Calculamos el número mayor que puede tener una posición.
        num_max = max(Hitori.tamaño_hor(estado),Hitori.tamaño_ver(estado))
        #Bucle de [0, tamaño vertical)
        for fila in range(0, Hitori.tamaño_ver(estado)):
            #Si algunos de los valores están dupliados en fila devolvemos verdadero
            for num in range(1, num_max+1):
                if estado.casillas[fila].count(num) > 1:
                    return True

        
    #Comprueba si existe algún duplicado en fila.    
    def duplicado_columna(self, estado):
        
        #Puntero de la columna
        for acHor in range(0, Hitori.tamaño_hor(estado)):
        
            #Puntero de la fila
            for acVer in range(0, Hitori.tamaño_ver(estado)):

                if Hitori.valor_casilla(estado, acVer, acHor) != 0:

                    for ac in range(acVer+1, Hitori.tamaño_ver(estado)):

                        if Hitori.valor_casilla(estado, acVer, acHor) == Hitori.valor_casilla(estado, ac, acHor):
                            return True   
    
    #Comprobamos que no existen duplicados en fila o en columna.
    def es_estado_final(self, estado):
        return bool(not self.duplicado_fila(estado) and not self.duplicado_columna(estado))

In [31]:
estado_inicial = Hitori([[4,5,2,1,9,6,7,8,6],[7,8,7,7,5,4,2,6,4],[3,9,1,2,1,5,6,5,8],[1,4,8,1,7,6,5,3,6],[6,1,1,5,8,3,2,7,9],[1,1,5,7,3,6,8,6,4],[4,6,4,8,7,1,5,9,8],[8,3,7,4,6,5,9,1,2],[8,2,6,7,1,8,9,1,5]])

In [32]:
p1 = ProblemaHitori(estado_inicial)

In [37]:
b_nueva = búsqee.BúsquedaEnProfundidad()

In [ ]:
%%timeit -n1 -r1

b_nueva.buscar(p1)